# Code to produce the plots of the paper

## 1. General Imports

In [104]:
import pandas as pd
import numpy as np

import scipy.stats as stats

import warnings

warnings.filterwarnings("ignore")

import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio


pio.templates["template"] = go.layout.Template()

pio.templates.default = "template"


adapter_list = [
    "dailymail",
    "wired",
    "express",
    "npr",
    "librarything",
    "instructables",
    "entrepreneur",
    "springer",
    "insiderpages",
    "ign",
    "eventbrite",
    "macrumors",
    "androidheadlines",
    "glassdoor",
    "pcworld",
    "csmonitor",
    "lonelyplanet",
    "booking",
    "journals",
    "frontiersin",
    "medium",
]

In [105]:
df = pd.read_excel("../data/experimental_results.xlsx")

## 2. Comparison Average & Ensemble

In [110]:
df_avg_ens[(df_avg_ens['topk'] < 1) & ~(df_avg_ens['combination_strategy'].isnull())]

,model,combination_strategy,weighting,eval_dataset,value,seed,chosen_adapters,topk,weights


In [111]:
df_avg_ens = df.copy()
df_avg_ens.loc[df_avg_ens["combination_strategy"].isnull(), "topk"] = 0


for model in df_avg_ens.model.unique():
    for index in df_avg_ens.loc[
        (df_avg_ens["model"] == model) & (df_avg_ens["combination_strategy"].isnull())
    ].index:
        for strat in ["average", "ensemble"]:
            for weighting in ["Sentence Sim", "TF-IDF", "Entropy", "Prior"]:
                df_avg_ens = df_avg_ens.append(
                    df_avg_ens.loc[[index] * 1].assign(
                        **{"weighting": weighting, "combination_strategy": strat}
                    )
                )


df_avg_ens["topk"] = df_avg_ens["topk"].fillna(21)


avg_seed = (
    df_avg_ens.groupby(
        by=["model", "combination_strategy", "weighting", "eval_dataset", "topk"]
    )
    .mean()
    .reset_index()
)

avg_dataset = (
    avg_seed.groupby(by=["model", "combination_strategy", "weighting", "topk"])
    .mean()
    .reset_index()
)

In [114]:
color_list = px.colors.qualitative.Set2


for model in avg_dataset.model.unique():
    fig = go.Figure()
    fig_difference = go.Figure()
    i = 0
    for weighting in ["Sentence Sim", "TF-IDF", "Entropy", "Prior"]:
        sub_ens = avg_dataset[
            (avg_dataset["weighting"] == weighting)
            & (avg_dataset["model"] == model)
            & (avg_dataset["combination_strategy"] == "ensemble")
        ]

        fig.add_trace(
            go.Scatter(
                x=sub_ens["topk"],
                y=sub_ens["value"],
                showlegend=False,
                marker=dict(size=8),
                line=dict(color=color_list[i], width=2, dash="dash"),
            )
        )

        sub_avg = avg_dataset[
            (avg_dataset["weighting"] == weighting)
            & (avg_dataset["model"] == model)
            & (avg_dataset["combination_strategy"] == "average")
        ]

        fig.add_trace(
            go.Scatter(
                x=sub_avg["topk"],
                y=sub_avg["value"],
                name=weighting,
                marker=dict(size=8),
                line=dict(color=color_list[i], width=2),
            )
        )

        df_difference = sub_ens[["topk", "value"]]
        df_difference.rename(columns={"value": "ensemble"}, inplace=True)
        df_difference
        df_difference = df_difference.merge(sub_avg, how="left", on="topk")
        df_difference["diff"] = df_difference["ensemble"] - df_difference["value"]

        df_difference = df_difference[df_difference['topk'] > 0]

        fig_difference.add_trace(
            go.Scatter(
                x=df_difference["topk"],
                y=df_difference["diff"],
                name=weighting,
                line=dict(color=color_list[i], width=2, dash="dot"),
            )
        )

        i += 1

    # Edit the layout
    fig.update_layout(
        title=model,
        xaxis_title="#-Adapters used",
        yaxis_title="Avg. Perplexity",
        font_family="Times New Roman",
        font=dict(size=24),
        legend=dict(
            yanchor="top",
            xanchor="right",
            bgcolor="rgba(0,0,0,0)",
            y=1.0,
            x=0.97,
        ),
    )

    fig.show()

    fig_difference.update_layout(
        title="Perplexity Difference " + model,
        xaxis_title="#-Adapters used",
        yaxis_title="Ensembling - Averaging PPL",
        font_family="Times New Roman",
        font=dict(size=24),
        legend=dict(
            yanchor="top",
            xanchor="right",
            y=1.015,
        ),
    )

    fig_difference.show()

## 3. Comparison weighted - unweighted

In [79]:
df_uni_weight = df[~df["weighting"].isnull()]

for weighting in ["Sentence sim", "TF-IDF", "Uncert", "prior"]:
    for index in df_uni_weight[df_uni_weight.weighting == "uniform"].index:
        df_uni_weight = df_uni_weight.append(
            df_uni_weight.loc[[index] * 1].assign(
                **{"weighting": weighting + "-topk-uniform"}
            )
        )
    df_uni_weight = df_uni_weight.reset_index(drop=True)


df_uni_weight.loc[
    df_uni_weight.weighting == "Entropy-topk-uniform", "weighting"
] = "Entropy Uniform"
df_uni_weight.loc[
    df_uni_weight.weighting == "uncert-topk-uniform", "weighting"
] = "Entropy Uniform"
df_uni_weight.loc[
    df_uni_weight.weighting == "Uncert-topk-uniform", "weighting"
] = "Entropy Uniform"
df_uni_weight.loc[df_uni_weight.weighting == "Uncert", "weighting"] = "Entropy"
df_uni_weight.loc[
    df_uni_weight.weighting == "tfidf-topk-uniform", "weighting"
] = "TF-IDF Uniform"
df_uni_weight.loc[
    df_uni_weight.weighting == "TF-IDF-topk-uniform", "weighting"
] = "TF-IDF Uniform"
df_uni_weight.loc[df_uni_weight.weighting == "prior", "weighting"] = "Prior"
df_uni_weight.loc[
    df_uni_weight.weighting == "Prior-topk-uniform", "weighting"
] = "Prior Uniform"
df_uni_weight.loc[
    df_uni_weight.weighting == "prior-topk-uniform", "weighting"
] = "Prior Uniform"
df_uni_weight.loc[
    df_uni_weight.weighting == "Sentence sim", "weighting"
] = "Sentence Sim"
df_uni_weight.loc[
    df_uni_weight.weighting == "Sentence Sim-topk-uniform", "weighting"
] = "Sentence Sim Uniform"
df_uni_weight.loc[
    df_uni_weight.weighting == "Sentence sim-topk-uniform", "weighting"
] = "Sentence Sim Uniform"


df_uni_weight = df_uni_weight[
    (df_uni_weight["model"].isin(["deberta", "gpt2"]))
    & (df_uni_weight["weighting"] != "uniform")
]

df_uni_weight = df_uni_weight[(df_uni_weight["topk"] != 1)]

In [78]:
df_uni_weight.weighting.unique()

array(['Sentence Sim', 'TF-IDF', 'Entropy', 'Prior',
       'Sentence Sim Uniform', 'TF-IDF Uniform', 'Entropy Uniform',
       'Prior Uniform', 'Uncert-topk-uniform'], dtype=object)

In [80]:
# Calculate SD
group_ds = (
    df_uni_weight.groupby(
        by=["model", "combination_strategy", "weighting", "seed", "topk"]
    )
    .mean()
    .reset_index()
)
grouped_data_total = group_ds.groupby(
    by=["model", "combination_strategy", "weighting", "topk"]
)["value"].agg([np.mean, np.std])
group_sizes = group_ds.groupby(
    by=["model", "combination_strategy", "weighting", "topk"]
).size()
standard_error = grouped_data_total["std"] / np.sqrt(group_sizes)
confidence_level = 0.95
margin_of_error = (
    stats.t.ppf((1 + confidence_level) / 2, group_sizes - 1) * standard_error
)
grouped_data_total["lower_bound"] = grouped_data_total["mean"] - margin_of_error
grouped_data_total["upper_bound"] = grouped_data_total["mean"] + margin_of_error
grouped_data_total = grouped_data_total.reset_index()

In [81]:
grouped_data_total = grouped_data_total[
    ~grouped_data_total.weighting.str.contains("Entropy")
]
grouped_data_total = grouped_data_total[
    ~grouped_data_total.weighting.str.contains("Prior")
]

In [82]:
grouped_data_total.model.unique()

array(['deberta', 'gpt2'], dtype=object)

In [83]:
grouped_data_total = grouped_data_total[
    ~grouped_data_total.weighting.str.contains("Entropy")
]
grouped_data_total = grouped_data_total[
    ~grouped_data_total.weighting.str.contains("Prior")
]

grouped_data_total.loc[
    grouped_data_total.weighting.str.contains("Uniform"), "weighting"
] = "uniform"
grouped_data_total.loc[
    grouped_data_total.weighting.str.contains("TF-IDF"), "weighting"
] = "weighted"
grouped_data_total.loc[
    grouped_data_total.weighting.str.contains("Sentence Sim"), "weighting"
] = "weighted"

grouped_data_total = (
    grouped_data_total.groupby(["model", "weighting", "topk"]).mean().reset_index()
)

color_list = px.colors.qualitative.Set2


template = "plotly_white"

for model in ["deberta", "gpt2"]:
    i = 2
    fig = go.Figure()

    tmp = grouped_data_total[
        (grouped_data_total["weighting"] == "weighted")
        & (grouped_data_total["model"] == model)
    ]

    fig.add_trace(
        go.Scatter(
            name="Weighted",
            x=tmp["topk"],
            y=tmp["mean"],
            mode="lines",
            line=dict(color="rgb(11, 188, 214)", width=2, dash="dash"),
        )
    )
    fig.add_trace(
        go.Scatter(
            name="Upper Bound",
            x=tmp["topk"],
            y=tmp["upper_bound"],
            mode="lines",
            marker=dict(color="rgb(11, 188, 214)"),
            line=dict(width=0),
            showlegend=False,
        )
    )
    fig.add_trace(
        go.Scatter(
            name="Lower Bound",
            x=tmp["topk"],
            y=tmp["lower_bound"],
            marker=dict(color="rgb(11, 188, 214)"),
            line=dict(width=0),
            mode="lines",
            fill="tonexty",
            showlegend=False,
        )
    )

    tmp_uni = grouped_data_total[
        (grouped_data_total["weighting"] == "uniform")
        & (grouped_data_total["model"] == model)
    ]

    fig.add_trace(
        go.Scatter(
            name="Unweighted",
            x=tmp_uni["topk"],
            y=tmp_uni["mean"],
            mode="lines",
            line=dict(color="rgb(230, 98, 94)", width=2),
        )
    )
    fig.add_trace(
        go.Scatter(
            name="Upper Bound Uni",
            x=tmp_uni["topk"],
            y=tmp_uni["upper_bound"],
            mode="lines",
            marker=dict(color="rgb(230, 98, 94)"),
            line=dict(width=0),
            showlegend=False,
        )
    )
    fig.add_trace(
        go.Scatter(
            name="Lower Bound Uni",
            x=tmp_uni["topk"],
            y=tmp_uni["lower_bound"],
            marker=dict(color="rgb(230, 98, 94)"),
            line=dict(width=0),
            mode="lines",
            fill="tonexty",
            showlegend=False,
        )
    )
    i += 1

    # Edit the layout
    fig.update_layout(
        title=model,
        xaxis_title="#-Adapters used",
        yaxis_title="Avg. Perplexity",
        font_family="Times New Roman",
        xaxis=dict(tickmode="linear", dtick=5),
        yaxis=dict(tickmode="linear", dtick=0.2),
        font=dict(size=30),
        legend=dict(yanchor="top", xanchor="left", x=0.02),
    )

    fig.show()

## 4. Efficiency

In [86]:
df_eff = pd.read_excel("../data/efficiency_calculcation.xlsx")

df_eff = df_eff[~df_eff["time"].isnull()]

df_eff_combined = df_eff.copy()
df_eff_combined.loc[
    df_eff_combined["calculcation"] == "Prior", "calculcation"
] = "Prior AVG"
df_eff_combined.loc[
    df_eff_combined["calculcation"] == "Entropy", "calculcation"
] = "Entropy AVG"
df_eff_combined.loc[
    df_eff_combined["calculcation"] == "TF-IDF", "calculcation"
] = "TF-IDF AVG"
df_eff_combined.loc[
    df_eff_combined["calculcation"] == "Sentence Sim", "calculcation"
] = "Sentence Sim AVG"

df_eff_combined.loc[df_eff_combined["model"] == "deberta", "model"] = "Deberta"
df_eff_combined.loc[df_eff_combined["model"] == "gpt2", "model"] = "GPT-2"


for calculation in ["Prior", "Entropy", "TF-IDF", "Sentence Sim"]:
    calc_string = calculation + " AVG"
    for index in df_eff_combined[df_eff_combined.calculcation == calc_string].index:
        df_eff_combined = df_eff_combined.append(
            df_eff_combined.loc[[index] * 1].assign(
                **{"calculcation": calculation + " ENS"}
            )
        )


for dataset in df_eff_combined.dataset.unique():
    for adapters in df_eff_combined.adapters.unique():
        for model in df_eff_combined.model.unique():
            for seed in df_eff_combined.Seed.unique():
                if (
                    len(
                        df_eff_combined[
                            (df_eff_combined.dataset == dataset)
                            & (df_eff_combined.adapters == adapters)
                            & (df_eff_combined.model == model)
                            & (df_eff_combined.Seed == seed)
                            & (df_eff_combined.calculcation == "Averaging")
                        ]["CO2 Emission"].values
                    )
                    == 1
                ):
                    avg_val = df_eff_combined[
                        (df_eff_combined.dataset == dataset)
                        & (df_eff_combined.adapters == adapters)
                        & (df_eff_combined.model == model)
                        & (df_eff_combined.Seed == seed)
                        & (df_eff_combined.calculcation == "Averaging")
                    ]["CO2 Emission"].values[0]

                    df_eff_combined.loc[
                        (df_eff_combined.dataset == dataset)
                        & (df_eff_combined.adapters == adapters)
                        & (df_eff_combined.model == model)
                        & (df_eff_combined.Seed == seed)
                        & (df_eff_combined.calculcation.str.contains("AVG")),
                        "CO2 Emission",
                    ] += avg_val

                if (
                    len(
                        df_eff_combined[
                            (df_eff_combined.dataset == dataset)
                            & (df_eff_combined.adapters == adapters)
                            & (df_eff_combined.model == model)
                            & (df_eff_combined.Seed == seed)
                            & (df_eff_combined.calculcation == "Ensembling")
                        ]["CO2 Emission"].values
                    )
                    == 1
                ):
                    avg_val = df_eff_combined[
                        (df_eff_combined.dataset == dataset)
                        & (df_eff_combined.adapters == adapters)
                        & (df_eff_combined.model == model)
                        & (df_eff_combined.Seed == seed)
                        & (df_eff_combined.calculcation == "Ensembling")
                    ]["CO2 Emission"].values[0]

                    df_eff_combined.loc[
                        (df_eff_combined.dataset == dataset)
                        & (df_eff_combined.adapters == adapters)
                        & (df_eff_combined.model == model)
                        & (df_eff_combined.Seed == seed)
                        & (df_eff_combined.calculcation.str.contains("ENS")),
                        "CO2 Emission",
                    ] += avg_val


grouped_data = df_eff_combined.groupby(by=["model", "calculcation", "adapters"])[
    "CO2 Emission"
].agg([np.mean, np.std])
group_sizes = df_eff_combined.groupby(by=["model", "calculcation", "adapters"]).size()
standard_error = grouped_data["std"] / np.sqrt(group_sizes)
confidence_level = 0.95
margin_of_error = (
    stats.t.ppf((1 + confidence_level) / 2, group_sizes - 1) * standard_error
)
grouped_data["lower_bound"] = grouped_data["mean"] - margin_of_error
grouped_data["upper_bound"] = grouped_data["mean"] + margin_of_error
grouped_data = grouped_data.reset_index()

df_eff_combined = (
    df_eff_combined.groupby(by=["model", "calculcation", "adapters"])
    .mean()
    .reset_index()
)

In [89]:
import re

color_list = px.colors.qualitative.Set2


template = "plotly_white"

for model in df_eff_combined.model.unique():
    fig = go.Figure()

    k = 0

    for weighting in ["Sentence Sim", "TF-IDF", "Entropy", "Prior"]:
        tmp_avg = grouped_data[
            (grouped_data.model == model)
            & (grouped_data.calculcation == weighting + " AVG")
        ]

        fig.add_trace(
            go.Scatter(
                x=tmp_avg["adapters"],
                y=tmp_avg["mean"],
                name=weighting,
                marker=dict(
                    size=10,
                ),
                line=dict(color=color_list[k], width=2),
            )
        )

        fig.add_trace(
            go.Scatter(
                name="Upper Bound",
                x=tmp_avg["adapters"],
                y=tmp_avg["upper_bound"],
                mode="lines",
                showlegend=False,
                marker=dict(color=color_list[k]),
                line=dict(width=0),
            )
        )
        # Create rgba value string
        rgb = color_list[k]
        parsed_rgb = re.search(r"(\d+),(\d+),(\d+)", rgb)
        rgba_string = (
            "rgba("
            + parsed_rgb[1]
            + ","
            + parsed_rgb[2]
            + ","
            + parsed_rgb[3]
            + ", 0.25)"
        )
        fig.add_trace(
            go.Scatter(
                name="Lower Bound",
                x=tmp_avg["adapters"],
                y=tmp_avg["lower_bound"],
                marker=dict(color=color_list[k]),
                line=dict(width=0),
                showlegend=False,
                fillcolor=rgba_string,
                mode="lines",
                fill="tonexty",
            )
        )

        tmp_ens = grouped_data[
            (grouped_data.model == model)
            & (grouped_data.calculcation == weighting + " ENS")
        ]

        fig.add_trace(
            go.Scatter(
                x=tmp_ens["adapters"],
                y=tmp_ens["mean"],
                name=weighting,
                showlegend=False,
                marker=dict(
                    size=10,
                ),
                line=dict(color=color_list[k], width=2, dash="dash"),
            )
        )

        fig.add_trace(
            go.Scatter(
                name="Upper Bound",
                x=tmp_ens["adapters"],
                y=tmp_ens["upper_bound"],
                mode="lines",
                showlegend=False,
                marker=dict(color=color_list[k]),
                line=dict(width=0),
            )
        )
        fig.add_trace(
            go.Scatter(
                name="Lower Bound",
                x=tmp_ens["adapters"],
                y=tmp_ens["lower_bound"],
                marker=dict(color=color_list[k]),
                line=dict(width=0),
                fillcolor=rgba_string,
                showlegend=False,
                mode="lines",
                fill="tonexty",
            )
        )

        fig.update_layout(
            title=model,
            font=dict(size=28),
            xaxis_title="#-Adapters",
            yaxis_title="CO₂ Emissions",
            font_family="Times New Roman",
            legend=dict(
                font=dict(size=24),
                yanchor="top",
                xanchor="left",
                x=0.015,
                y=1.015,
                traceorder="normal",
                bgcolor="rgba(0,0,0,0)",
            ),
        )

        k += 1

    fig.show()

## 5. Weight Comparison

In [96]:
comp_weights_df = df[
    (df["topk"] == 21)
    & (df["weighting"].isin(["uniform", "Sentence Sim", "TF-IDF", "Entropy", "Prior"]))
]

for adapter in adapter_list:
    comp_weights_df[adapter] = 0


def get_weight_of_adapter(row, i):
    weight = 0.0
    if isinstance(row["weights"], str):
        weight_str = row["weights"].split("-")
        try:
            weight = float(weight_str[i])
        except:
            print(row)
    elif row["weighting"] == "uniform" and row["topk"] == 21:
        weight = float(0.047619047619047616)
    return weight


for i in range(len(adapter_list)):
    comp_weights_df[adapter_list[i]] = comp_weights_df.apply(
        lambda row: get_weight_of_adapter(row, i), axis=1
    )

grouped_df = comp_weights_df.groupby(by=['model', 'combination_strategy', 'weighting', 'eval_dataset']).mean().reset_index()
grouped_df = grouped_df.groupby(by=['model', 'combination_strategy', 'weighting']).mean().reset_index()
grouped_df = grouped_df.groupby(by=['model', 'weighting']).mean().reset_index()

In [99]:
import plotly.express as px
import pandas as pd

for model in grouped_df.model.unique():

    sub = grouped_df[grouped_df['model'] == model][['weighting','dailymail', 'wired',
        'express', 'npr', 'librarything', 'instructables', 'entrepreneur',
        'springer', 'insiderpages', 'ign', 'eventbrite', 'macrumors',
        'androidheadlines', 'glassdoor', 'pcworld', 'csmonitor', 'lonelyplanet',
        'booking', 'journals', 'frontiersin', 'medium']]
    sub = sub.transpose()
    sub.rename(columns=sub.iloc[0], inplace = True)
    sub.drop(sub.index[0], inplace = True)
    sub = sub.reset_index()
    sub.rename(columns={"index": "adapter"}, inplace = True)

    fig = go.Figure()

    fig.add_trace(go.Scatterpolar(
        r=sub['Sentence Sim'],
        theta=sub.adapter,
        name='Sentence Sim',
        line_color = color_list[0]
    ))
    fig.add_trace(go.Scatterpolar(
        r=sub['TF-IDF'],
        theta=sub.adapter,
        name='TF-IDF',
        line_color = color_list[1]
    ))
    fig.add_trace(go.Scatterpolar(
        r=sub['Entropy'],
        theta=sub.adapter,
        name='Entropy',
        line_color = color_list[2]
    ))
    fig.add_trace(go.Scatterpolar(
        r=sub['Prior'],
        theta=sub.adapter,
        name='Prior',
        line_color = color_list[3]
    ))
    fig.add_trace(go.Scatterpolar(
        r=sub['uniform'],
        theta=sub.adapter,
        fill='toself',
        name='Uniform',
        line_color = color_list[7],
        opacity=.5
    ))

    fig.update_layout(
        title=model,
        font_family="Times New Roman",
                    font=dict(size=22),
        legend=dict(
                            yanchor="top",
                            xanchor="right",
                        y=0.95,
                        x=1.65,
            bgcolor = 'rgba(0,0,0,0)'
                        ),    
        showlegend=False,
    polar=dict(
        radialaxis=dict(
        visible=True,
        range=[0, 0.08]
        ))
    )
    fig.show()